In [1]:
import pandas as pd

In [15]:
df = pd.read_csv('../data/electricity-generation_emissions_sources_v4_4_0.csv')

In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 489466 entries, 0 to 489465
Data columns (total 47 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   source_id               489466 non-null  int64  
 1   source_name             489466 non-null  object 
 2   source_type             489466 non-null  object 
 3   iso3_country            489466 non-null  object 
 4   sector                  489466 non-null  object 
 5   subsector               489466 non-null  object 
 6   start_time              489466 non-null  object 
 7   end_time                489466 non-null  object 
 8   lat                     489466 non-null  float64
 9   lon                     489466 non-null  float64
 10  geometry_ref            0 non-null       float64
 11  gas                     489466 non-null  object 
 12  emissions_quantity      489466 non-null  float64
 13  temporal_granularity    489466 non-null  object 
 14  activity            

In [19]:
df.isnull().sum()

source_id                      0
source_name                    0
source_type                    0
iso3_country                   0
sector                         0
subsector                      0
start_time                     0
end_time                       0
lat                            0
lon                            0
geometry_ref              489466
gas                            0
emissions_quantity             0
temporal_granularity           0
activity                       0
activity_units                 0
emissions_factor               0
emissions_factor_units         0
capacity                       0
capacity_units                 0
capacity_factor                0
other1                         0
other1_def                     0
other2                         0
other2_def                     0
other3                         0
other3_def                     0
other4                         0
other4_def                     0
other5                         0
other5_def

In [25]:
df.dropna(axis=1, inplace=True)

In [27]:
df.head().T

,0,1,2,3,4
source_id,25452484,25452484,25452484,25452484,25452484
source_name,BALASHI,BALASHI,BALASHI,BALASHI,BALASHI
source_type,oil,oil,oil,oil,oil
iso3_country,ABW,ABW,ABW,ABW,ABW
sector,power,power,power,power,power
subsector,electricity-generation,electricity-generation,electricity-generation,electricity-generation,electricity-generation
start_time,2021-01-01 00:00:00,2021-02-01 00:00:00,2021-03-01 00:00:00,2021-04-01 00:00:00,2021-05-01 00:00:00
end_time,2021-01-31 00:00:00,2021-02-28 00:00:00,2021-03-31 00:00:00,2021-04-30 00:00:00,2021-05-31 00:00:00
lat,12.4759,12.4759,12.4759,12.4759,12.4759
lon,-69.9823,-69.9823,-69.9823,-69.9823,-69.9823


In [31]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

In [33]:
spark = SparkSession.builder.appName("esg").getOrCreate()
spark

In [73]:
df = spark.read.csv('../data/electricity-generation_emissions_sources_v4_4_0.csv', header=True, inferSchema=True)

In [74]:
df.printSchema()

root
 |-- source_id: integer (nullable = true)
 |-- source_name: string (nullable = true)
 |-- source_type: string (nullable = true)
 |-- iso3_country: string (nullable = true)
 |-- sector: string (nullable = true)
 |-- subsector: string (nullable = true)
 |-- start_time: timestamp (nullable = true)
 |-- end_time: timestamp (nullable = true)
 |-- lat: double (nullable = true)
 |-- lon: double (nullable = true)
 |-- geometry_ref: string (nullable = true)
 |-- gas: string (nullable = true)
 |-- emissions_quantity: double (nullable = true)
 |-- temporal_granularity: string (nullable = true)
 |-- activity: double (nullable = true)
 |-- activity_units: string (nullable = true)
 |-- emissions_factor: double (nullable = true)
 |-- emissions_factor_units: string (nullable = true)
 |-- capacity: double (nullable = true)
 |-- capacity_units: string (nullable = true)
 |-- capacity_factor: double (nullable = true)
 |-- other1: string (nullable = true)
 |-- other1_def: string (nullable = true)
 |--

In [107]:
from pyspark.sql.functions import col

df.filter(col('other6').isNull()).show()

+---------+-----------+-----------+------------+------+--------------------+-------------------+-------------------+-------+--------+------------+----------+------------------+--------------------+--------+--------------+------------------+----------------------+--------+--------------+------------------+------+--------------------+------+----------------+------+------------------+------+-----------------+------+---------------+------+----------+------+--------------------+------+----------+------+----------+-------+-----------+-------------------+--------------------+----------------+-------+---------+----------------+
|source_id|source_name|source_type|iso3_country|sector|           subsector|         start_time|           end_time|    lat|     lon|geometry_ref|       gas|emissions_quantity|temporal_granularity|activity|activity_units|  emissions_factor|emissions_factor_units|capacity|capacity_units|   capacity_factor|other1|          other1_def|other2|      other2_def|other3|       

In [95]:
from pyspark.sql.functions import col, sum

null_counts = df.select([sum(col(c).isNull().cast('int')).alias(c) for c in df.columns])
null_counts.show()

+---------+-----------+-----------+------------+------+---------+----------+--------+---+---+------------+---+------------------+--------------------+--------+--------------+----------------+----------------------+--------+--------------+---------------+------+----------+------+----------+------+----------+------+----------+------+----------+------+----------+------+----------+------+----------+------+----------+-------+-----------+------------+-------------+----------------+-------+---------+----------------+
|source_id|source_name|source_type|iso3_country|sector|subsector|start_time|end_time|lat|lon|geometry_ref|gas|emissions_quantity|temporal_granularity|activity|activity_units|emissions_factor|emissions_factor_units|capacity|capacity_units|capacity_factor|other1|other1_def|other2|other2_def|other3|other3_def|other4|other4_def|other5|other5_def|other6|other6_def|other7|other7_def|other8|other8_def|other9|other9_def|other10|other10_def|created_date|modified_date|reporting_entity|lat_

In [65]:
total_rows = df.count()
print(f"Total rows in DataFrame: {total_rows}")

# Calculate null counts for each column
null_counts = df.select([count(when(col(c).isNull(), c)).alias(c) for c in df.columns]).collect()[0].asDict()

# Identify columns to drop (where null count equals total_rows)
cols_to_drop_all_null = [c for c, count in null_counts.items() if count == total_rows]

print(f"Columns with all null values to drop: {cols_to_drop_all_null}")

# Drop the identified columns
df_dropped_all_null = df.drop(*cols_to_drop_all_null)
print("DataFrame after dropping columns with all null values:")
df_dropped_all_null.show()


Total rows in DataFrame: 489466
Columns with all null values to drop: ['geometry_ref', 'other6', 'other6_def', 'other8', 'other8_def', 'other9', 'other9_def', 'other10', 'other10_def', 'lat_lon']
DataFrame after dropping columns with all null values:
+---------+-----------+-----------+------------+------+--------------------+-------------------+-------------------+-------+--------+----------+------------------+--------------------+--------+--------------+------------------+----------------------+--------+--------------+------------------+------+--------------------+------+----------------+------+------------------+------+-----------------+------+---------------+------+--------------------+-------------------+--------------------+----------------+---------+----------------+
|source_id|source_name|source_type|iso3_country|sector|           subsector|         start_time|           end_time|    lat|     lon|       gas|emissions_quantity|temporal_granularity|activity|activity_units|  emissi

In [67]:
df_dropped_all_null.count()

489466

In [69]:
df_dropped_all_null.printSchema()

root
 |-- source_id: string (nullable = true)
 |-- source_name: string (nullable = true)
 |-- source_type: string (nullable = true)
 |-- iso3_country: string (nullable = true)
 |-- sector: string (nullable = true)
 |-- subsector: string (nullable = true)
 |-- start_time: string (nullable = true)
 |-- end_time: string (nullable = true)
 |-- lat: string (nullable = true)
 |-- lon: string (nullable = true)
 |-- gas: string (nullable = true)
 |-- emissions_quantity: string (nullable = true)
 |-- temporal_granularity: string (nullable = true)
 |-- activity: string (nullable = true)
 |-- activity_units: string (nullable = true)
 |-- emissions_factor: string (nullable = true)
 |-- emissions_factor_units: string (nullable = true)
 |-- capacity: string (nullable = true)
 |-- capacity_units: string (nullable = true)
 |-- capacity_factor: string (nullable = true)
 |-- other1: string (nullable = true)
 |-- other1_def: string (nullable = true)
 |-- other2: string (nullable = true)
 |-- other2_def: 

In [71]:
null_counts

{'source_id': 0,
 'source_name': 0,
 'source_type': 0,
 'iso3_country': 0,
 'sector': 0,
 'subsector': 0,
 'start_time': 0,
 'end_time': 0,
 'lat': 0,
 'lon': 0,
 'geometry_ref': 489466,
 'gas': 0,
 'emissions_quantity': 0,
 'temporal_granularity': 0,
 'activity': 0,
 'activity_units': 0,
 'emissions_factor': 0,
 'emissions_factor_units': 0,
 'capacity': 0,
 'capacity_units': 0,
 'capacity_factor': 0,
 'other1': 0,
 'other1_def': 0,
 'other2': 0,
 'other2_def': 0,
 'other3': 0,
 'other3_def': 0,
 'other4': 0,
 'other4_def': 0,
 'other5': 0,
 'other5_def': 0,
 'other6': 489466,
 'other6_def': 489466,
 'other7': 489373,
 'other7_def': 0,
 'other8': 489466,
 'other8_def': 489466,
 'other9': 489466,
 'other9_def': 489466,
 'other10': 489466,
 'other10_def': 489466,
 'created_date': 0,
 'modified_date': 0,
 'reporting_entity': 0,
 'lat_lon': 489466,
 'sector_id': 0,
 'native_source_id': 0}